# FEMA's National Flood Insurance Policy Database Data Exploration

### Load Libraries

In [21]:
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.ml.stat import Correlation
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Imputer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, ByteType, LongType, FloatType, ShortType
from pyspark.sql.functions import col, sum as spark_sum

### Initialize Spark Session

In [2]:
spark = SparkSession.builder \
    .config("spark.driver.memory", "10g") \
	.config("spark.executor.memory", "2g") \
    .config('spark.executor.instances', 5) \
	.config("spark.sql.debug.maxToStringFields", "100")\
    .appName("Flood Data") \
	.getOrCreate()

24/05/28 12:55:30 WARN Utils: Your hostname, MK-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.39 instead (on interface en0)
24/05/28 12:55:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/28 12:55:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/28 12:55:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Load data to Spark dataframe; Infer Schema

In [3]:
#path to data 
data_path = "./NFIP/nfip-flood-policies.csv"

df = spark.read.option("header","true").option("inferSchema","true").csv(data_path)

24/05/28 12:55:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Perform the Data Exploration
Below is the outline of the Data Exploration Section:
- Print Schema
- Column Description
- Display the Number of Variables (Columnns)
- Show the First Few Rows
- Display Number of Observations
- Missing Values in the Dataframe
- Statistics Summary and Data Distribution
- Correlations Among Variables
- Scatterplots

### Print Schema

Understanding the structure of data (column names and types) is crucial.

In [4]:
df.printSchema()

root
 |-- agriculturestructureindicator: string (nullable = true)
 |-- basefloodelevation: double (nullable = true)
 |-- basementenclosurecrawlspacetype: integer (nullable = true)
 |-- cancellationdateoffloodpolicy: date (nullable = true)
 |-- censustract: long (nullable = true)
 |-- condominiumindicator: string (nullable = true)
 |-- construction: string (nullable = true)
 |-- countycode: integer (nullable = true)
 |-- crsdiscount: double (nullable = true)
 |-- deductibleamountinbuildingcoverage: string (nullable = true)
 |-- deductibleamountincontentscoverage: string (nullable = true)
 |-- elevatedbuildingindicator: string (nullable = true)
 |-- elevationcertificateindicator: string (nullable = true)
 |-- elevationdifference: integer (nullable = true)
 |-- federalpolicyfee: integer (nullable = true)
 |-- floodzone: string (nullable = true)
 |-- hfiaasurcharge: integer (nullable = true)
 |-- houseofworshipindicator: string (nullable = true)
 |-- latitude: double (nullable = true)
 |--

### Column Description

**Description of the columns from the FEMA's National Flood Insurance Policy Database, grouped by their data types and purpose:**

**Geographic and Location Details**
- **censustract (long):** Census tract number indicating the specific area where the property is located, used for demographic analysis.
- **countycode (integer):** Numeric code representing the county in which the property is insured.
- **floodzone (string):** Designation of the flood zone according to FEMA's mapping, crucial for assessing the property's flood risk.
- **latitude (double), longitude (double):** Geographic coordinates specifying the precise location of the property.
- **propertystate (string):** The U.S. state where the property is located.
- **reportedcity (string):** The city reported for the insured property.
- **reportedzipcode (integer):** Zip code where the property is situated, used for localizing insurance coverage and risk.

**Property and Construction Details**
- **agriculturestructureindicator (string):** Indicates whether the property is used for agricultural purposes.
- **basementenclosurecrawlspacetype (integer):** Type of basement or crawlspace present at the property, affecting flood risk assessment.
- **construction (string):** Describes the type of construction materials and methods used, which can affect the property's vulnerability to flood damage.
- **numberoffloorsininsuredbuilding (integer):** Total floors in the insured building, important for determining potential flood damage and insurance coverage needs.
- **elevatedbuildingindicator (string):** Indicates whether the building is elevated, a key factor in reducing flood risk.

**Policy Details**
- **policycost (integer):** The total cost of the flood insurance policy.
- **policycount (integer):** The number of policies associated with a single property or account.
- **policyeffectivedate (date), policyterminationdate (date):** Start and end dates of the flood insurance coverage.
- **totalbuildinginsurancecoverage (integer), totalcontentsinsurancecoverage (integer):** The amount of insurance coverage for the building and its contents, respectively.
- **totalinsurancepremiumofthepolicy (integer):** Total premium amount for the flood insurance policy.

**Flood Risk Assessment Specifics**
- **basefloodelevation (double):** The base flood elevation expected for a particular area, critical for understanding flood risk levels.
- **elevationcertificateindicator (string), elevationdifference (integer):** Presence of an elevation certificate and the difference in elevation, respectively, both crucial for assessing compliance with floodplain management.
- **lowestadjacentgrade (double), lowestfloorelevation (double):** Measures of elevation that help determine the property's flood exposure.

**Insurance Policy Features**
- **crsdiscount (double):** Community Rating System discount applied to the policy, which can reduce insurance premiums based on community flood preparedness.
- **deductibleamountinbuildingcoverage (integer), deductibleamountincontentscoverage (integer):** Deductible amounts for building and contents coverage, influencing out-of-pocket costs after a flood.
- **hfiaasurcharge (integer):** Surcharge applied under the Homeowner Flood Insurance Affordability Act.
- **federalpolicyfee (integer):** A fee associated with the federal policy governing flood insurance.

**Special Indicators**
- **condominiumindicator (string), primaryresidenceindicator (string):** Indicate whether the insured property is a condominium or the primary residence of the owner.
- **houseofworshipindicator (string), nonprofitindicator (string):** Indicators of whether the property is used as a house of worship or is owned by a nonprofit organization, affecting policy terms and possibly qualifying for special considerations.
- **postfirmconstructionindicator (string):** Indicates if the building was constructed after the community's first Flood Insurance Rate Map was issued, which can affect insurance rates.
- **smallbusinessindicatorbuilding (string):** Indicates whether the insured building is used for small business purposes.

**Additional Policy and Coverage Information**
- **originalconstructiondate (date), originalnbdate (date):** Dates of original construction and the building's initial notebook entry, important for historical property assessments.
- **cancellationdateoffloodpolicy (date):** Date when the flood policy was cancelled, if applicable.
- **regularemergencyprogramindicator (string):** Indicates the type of FEMA program under which the policy is covered, distinguishing between regular and emergency management programs.
- **ratemethod (integer):** Describes the method used to calculate the insurance rate, impacting how premiums are determined.
- **locationofcontents (string):** Specifies where within the property the insured contents are located, relevant for claims and risk assessments.

### Display number of Variables (Columns)


In [5]:
num_variables = len(df.columns)
print("Number of Columns:", num_variables)

Number of Columns: 45


### Show the First Few Rows

In [6]:
df.show(5,vertical=True)

-RECORD 0--------------------------------------------------
 agriculturestructureindicator      | NULL                 
 basefloodelevation                 | NULL                 
 basementenclosurecrawlspacetype    | 2                    
 cancellationdateoffloodpolicy      | NULL                 
 censustract                        | 33013038500          
 condominiumindicator               | N                    
 construction                       | N                    
 countycode                         | 33013                
 crsdiscount                        | 0.0                  
 deductibleamountinbuildingcoverage | 0                    
 deductibleamountincontentscoverage | 0                    
 elevatedbuildingindicator          | N                    
 elevationcertificateindicator      | NULL                 
 elevationdifference                | 999                  
 federalpolicyfee                   | 13                   
 floodzone                          | X 

### Display Number of Observations

In [7]:
df.count()

50406943

FEMA's National Flood Insurance Policy Database, containing over 50 million (50,406,943) policy transactions.

### Missing Values in the Dataframe

In [5]:
#find number of missing values in the dataframe
missing_vals = df.select(*(spark_sum(col(i).isNull().cast("int")).alias(i) for i in df.columns))

In [6]:
missing_vals.show(vertical= True)

-RECORD 0--------------------------------------
 agriculturestructureindicator      | 38923313 
 basefloodelevation                 | 33636759 
 basementenclosurecrawlspacetype    | 802      
 cancellationdateoffloodpolicy      | 43614057 
 censustract                        | 467119   
 condominiumindicator               | 6        
 construction                       | 13       
 countycode                         | 48999    
 crsdiscount                        | 0        
 deductibleamountinbuildingcoverage | 661993   
 deductibleamountincontentscoverage | 5561584  
 elevatedbuildingindicator          | 258      
 elevationcertificateindicator      | 32606397 
 elevationdifference                | 0        
 federalpolicyfee                   | 0        
 floodzone                          | 169145   
 hfiaasurcharge                     | 0        
 houseofworshipindicator            | 34476251 
 latitude                           | 338699   
 locationofcontents                 | 15

**Here's a description of the missing value situation in the FEMA Flood Insurance Policy Database:** 

1. **High Missing Values:**
- **Base Flood Elevation, Latitude, Longitude, Lowest Adjacent Grade, Lowest Floor Elevation:** These fields each have around 50,406,943 missing values. This suggests a significant lack of geographic and elevation data, which are critical in flood insurance calculations.
- **Elevation Certificate Indicator, Elevation Difference:** Both fields are missing approximately 32,806,397 and 32,897,994 values respectively, indicating that elevation certificates, which are vital for verifying compliance with floodplain management regulations, are largely absent.
- **Obstruction Type:** Missing around 40,793,526 values, indicating that details about obstructions which can affect flood risk assessments are predominantly not reported.
- **House of Worship Indicator, Nonprofit Indicator:** Each has over 34,476,251 and 34,493,094 missing entries respectively, indicating a lack of identification of these property types, which might have different considerations in policy terms.

2. **Moderate Missing Values:**
- **Deductible Amount in Building Coverage, Deductible Amount in Contents Coverage:** Missing 15,649,149 and 18,265,104 values respectively, which implies incomplete data on policy deductibles that could affect premium calculations and risk assessments.
- **Location of Contents:** With 15,389,767 missing entries, there's substantial missing information on where contents are located within the insured buildings, which is vital for damage assessments.

3. **Low Missing Values:**
- **Census Tract, Flood Zone:** Missing 467,119 and 169,145 entries respectively. Although relatively lower, these still represent significant gaps, especially as these fields are crucial for location-specific risk assessment.
- **Number of Floors in Insured Building:** Missing data on 162,301 entries could affect understanding building structure and associated risk.

4. **Minimal to No Missing Values:**
- Fields like **CRS Discount, Federal Policy Fee**, and various policy-related dates (effectiveness, termination) and costs show zero missing values, indicating complete data in terms of policy transaction details.
- Similarly, **County Code, Construction, Condominium Indicator, Occupancy Type** show minimal missing data (under 50,000), suggesting good coverage of basic property and policyholder information.

Overall, the dataset shows a strong presence of policy and basic property information but suffers from a significant absence of detailed geographic and structural data. This gap in data can hamper effective risk assessment and pricing of flood insurance policies, especially in areas prone to flooding where such data is most critical. Addressing these missing values, either by data imputation where appropriate or by collecting missing data, could significantly enhance the robustness of any analysis or predictive modeling based on this dataset.

### Statistics Summary and Data Distribution

In [10]:
numerical_columns = [col_name for col_name, data_type in df.dtypes if data_type in ["int", "bigint", "double"]]

# Select only numerical columns
numerical_df = df.select(*numerical_columns)

# Generate summary statistics
summary_stats = numerical_df.describe()

# Show summary statistics
summary_stats.show(vertical = True)

24/05/28 13:00:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0------------------------------------------------
 summary                          | count                
 basefloodelevation               | 16770184             
 basementenclosurecrawlspacetype  | 50406141             
 censustract                      | 49939824             
 countycode                       | 50357944             
 crsdiscount                      | 50406943             
 elevationdifference              | 50406943             
 federalpolicyfee                 | 50406943             
 hfiaasurcharge                   | 50406943             
 latitude                         | 50068244             
 longitude                        | 50068244             
 lowestadjacentgrade              | 15466364             
 lowestfloorelevation             | 17346341             
 numberoffloorsininsuredbuilding  | 50244642             
 occupancytype                    | 50406937             
 policycost                       | 50406943             
 policycount  

The summary statistics for the FEMA National Flood Insurance Policy Database provide a comprehensive overview of various policy and property-related numerical attributes. These statistics include measures of central tendency, dispersion, and range, all of which are critical for understanding the distribution and potential data quality issues within the dataset. Below is a detailed analysis of the key statistical summaries:

**Central Tendency and Dispersion**
1. **Base Flood Elevation:**
- Average (Mean): 119.47 ft.
- Standard Deviation: 522.49 ft.
- Range: -9999 - 85,640 ft.
2. **Lowest Adjacent Grade:**
- Average (Mean): 129.20 ft.
- Standard Deviation: 609.92 ft.
- Range: -9,999 - 99,990.9 ft.
3. **Lowest Floor Elevation:**
- Average (Mean): 385.62 ft.
- Standard Deviation: 1,676.42 ft.
- Range: -9,997.9 - 99,989 ft.
4. **Basement Enclosure Crawl Space Type:**
- Average (Mean): 0.37, indicating a slight bias towards lower classifications.
- Standard Deviation: 0.86, showing moderate variability within the data.
- Range: Min 0 to Max 4, spanning several classification levels.
5. **Census Tract:**
- Average (Mean): Approximately 2.6 x 10¹⁰.
- Standard Deviation: About 1.58 x 10¹⁰, suggesting a wide spread across census tracts.
6. **CRS Discount:**
- Average: 0.064, typically low across the dataset.
- Standard Deviation: 0.091, with most data points close to zero but some higher values.
7. **Deductible Amount in Building and Contents Coverage:**
- Building Coverage Average: 1.66 with a deviation of 1.46.
- Contents Coverage Average: 0.98 with a deviation of 1.05.
- Both show low average deductible amounts but with notable variation.
8. **Elevation Difference:**
- Average: 1.69, indicating minor differences in elevation on average.
- Standard Deviation: 3.39, suggesting significant outliers affecting the elevation difference.
9. **Policy Related Figures (Policy Cost, Policy Count, Total Insurance Coverage, etc.):**
- These values have a high mean and standard deviation, indicating a significant spread in the policy costs and coverages, reflecting diverse insurance policies and property valuations.

**Extremes (Minimum and Maximum Values)**
- Notable minimums include negative values in **Federal Policy Fee and HFIAA Surcharge**, possibly indicating refunds or adjustments.
- The maximum values in **Total Building Insurance Coverage and Total Insurance Premium of the Policy** reach into the hundreds of millions, highlighting cases with exceptionally high insurance coverage.

**Implications**
The substantial missing data in critical geographical and elevation columns could significantly hinder risk assessment accuracy. The wide variability in policy costs and coverage levels underscores the diverse nature of the insured properties. Accurate and complete data in these fields are crucial for effective risk management and policy pricing in flood insurance.

This analysis provides a basis for further data cleaning, particularly in addressing missing values and outliers, which are essential for improving data quality and the reliability of subsequent analyses and decision-making processes based on this dataset.

### Correlations Among Variables

In [11]:
numerical_df.show(5,vertical=True)

-RECORD 0---------------------------------------
 basefloodelevation               | NULL        
 basementenclosurecrawlspacetype  | 2           
 censustract                      | 33013038500 
 countycode                       | 33013       
 crsdiscount                      | 0.0         
 elevationdifference              | 999         
 federalpolicyfee                 | 13          
 hfiaasurcharge                   | 0           
 latitude                         | 43.3        
 longitude                        | -71.8       
 lowestadjacentgrade              | NULL        
 lowestfloorelevation             | NULL        
 numberoffloorsininsuredbuilding  | 2           
 occupancytype                    | 1           
 policycost                       | 388         
 policycount                      | 1           
 policytermindicator              | 1           
 totalbuildinginsurancecoverage   | 250000      
 totalcontentsinsurancecoverage   | 100000      
 totalinsurancepremi

In [12]:
v_col = "features"
input = numerical_df.columns
output = [v_col + str(i) for i in range(len(input))]
imputer = Imputer(strategy='mean',inputCols=input,outputCols=output)
changed_df = imputer.fit(numerical_df).transform(numerical_df) 

assembler = VectorAssembler(inputCols=output,outputCol=v_col)
numerical_df_vector = assembler.transform(changed_df).select(v_col)
numerical_df_vector

DataFrame[features: vector]

In [ ]:
matrix = Correlation.corr(numerical_df_vector,v_col).collect()[0][0]
corr_matrix = matrix.toArray().tolist()
print(corr_matrix)

In [ ]:
numerical_df.columns

In [ ]:
columns = ['basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy']
df_c = spark.createDataFrame(corr_matrix,columns)

In [17]:
df_c.select('basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract').show(20)

24/05/28 11:05:45 ERROR Executor: Exception in task 0.0 in stage 26.0 (TID 715)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/mengkong/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 11) than that in driver 3.9, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org

Py4JJavaError: An error occurred while calling o444.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 715) (192.168.4.39 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/mengkong/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 11) than that in driver 3.9, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at 

In [18]:
df_c.select('countycode',
 'crsdiscount',
 'elevationdifference').show(20)

24/05/28 11:05:48 ERROR Executor: Exception in task 0.0 in stage 27.0 (TID 716)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/mengkong/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 11) than that in driver 3.9, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org

Py4JJavaError: An error occurred while calling o481.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 1 times, most recent failure: Lost task 0.0 in stage 27.0 (TID 716) (192.168.4.39 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/mengkong/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 11) than that in driver 3.9, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at 

In [ ]:
df_c.select('federalpolicyfee',
 'hfiaasurcharge',
 'latitude').show(20)

In [ ]:
df_c.select('longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation').show(20)

In [ ]:
df_c.select('numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost').show(20)

In [ ]:
df_c.select('policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage').show(20)

In [ ]:
df_c.select('totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy').show(20)

The dataframe new_df is the dataframe with the imputed mean values and this code checks that there are zero missing values and the column names are changed from feature0, feature1 and so on to the original column names within the new_df dataframe.

In [37]:
new_df = changed_df.drop('basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy')
new_df.show(5,vertical=True)

-RECORD 0------------------------
 features0  | 119.46875184553842 
 features1  | 2                  
 features2  | 33013038500        
 features3  | 33013              
 features4  | 0.0                
 features5  | 999                
 features6  | 13                 
 features7  | 0                  
 features8  | 43.3               
 features9  | -71.8              
 features10 | 129.1970600006573  
 features11 | 385.6220114835789  
 features12 | 2                  
 features13 | 1                  
 features14 | 388                
 features15 | 1                  
 features16 | 1                  
 features17 | 250000             
 features18 | 100000             
 features19 | 375                
-RECORD 1------------------------
 features0  | 119.46875184553842 
 features1  | 0                  
 features2  | 22063040700        
 features3  | 22063              
 features4  | 0.05               
 features5  | 999                
 features6  | 35                 
 features7  | 

In [38]:
new_df_missing_vals = new_df.select(*(spark_sum(col(i).isNull().cast("int")).alias(i) for i in new_df.columns))

In [39]:
new_df_missing_vals.show(vertical= True)

-RECORD 0---------
 features0  | 0   
 features1  | 0   
 features2  | 0   
 features3  | 0   
 features4  | 0   
 features5  | 0   
 features6  | 0   
 features7  | 0   
 features8  | 0   
 features9  | 0   
 features10 | 0   
 features11 | 0   
 features12 | 0   
 features13 | 0   
 features14 | 0   
 features15 | 0   
 features16 | 0   
 features17 | 0   
 features18 | 0   
 features19 | 0   



In [40]:
column_names = ['basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy']
new_df = new_df.toDF(*column_names)
new_df.show(5,vertical=True)

-RECORD 0----------------------------------------------
 basefloodelevation               | 119.46875184553842 
 basementenclosurecrawlspacetype  | 2                  
 censustract                      | 33013038500        
 countycode                       | 33013              
 crsdiscount                      | 0.0                
 elevationdifference              | 999                
 federalpolicyfee                 | 13                 
 hfiaasurcharge                   | 0                  
 latitude                         | 43.3               
 longitude                        | -71.8              
 lowestadjacentgrade              | 129.1970600006573  
 lowestfloorelevation             | 385.6220114835789  
 numberoffloorsininsuredbuilding  | 2                  
 occupancytype                    | 1                  
 policycost                       | 388                
 policycount                      | 1                  
 policytermindicator              | 1           

In [41]:
print(new_df.dtypes)

[('basefloodelevation', 'double'), ('basementenclosurecrawlspacetype', 'int'), ('censustract', 'bigint'), ('countycode', 'int'), ('crsdiscount', 'double'), ('elevationdifference', 'int'), ('federalpolicyfee', 'int'), ('hfiaasurcharge', 'int'), ('latitude', 'double'), ('longitude', 'double'), ('lowestadjacentgrade', 'double'), ('lowestfloorelevation', 'double'), ('numberoffloorsininsuredbuilding', 'int'), ('occupancytype', 'int'), ('policycost', 'int'), ('policycount', 'int'), ('policytermindicator', 'int'), ('totalbuildinginsurancecoverage', 'int'), ('totalcontentsinsurancecoverage', 'int'), ('totalinsurancepremiumofthepolicy', 'int')]


In [42]:
# Milestone 3: Construct Linear Regression model to perform Feature Importance (find weights of features, choose top weights' features to keep)
# creating train and test sets
# X = new_df.drop('totalinsurancepremiumofthepolicy')
# y = df['totalinsurancepremiumofthepolicy']
# ASK: Is 'features' correct?
assembler = VectorAssembler(
    inputCols=['basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage'],
    outputCol = 'features')
new_df = assembler.transform(new_df)
final_data = new_df.select('features', 'totalinsurancepremiumofthepolicy')

train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=42)

lr = LinearRegression(featuresCol = 'features', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium', regParam = 0.3)
lr_model = lr.fit(train_data)

In [43]:
# Milestone 3: Print coefficients of the features (will run this cell AFTER LinReg is successful)
coefficients = lr_model.coefficients
print("Coefficients: {}".format(lr_model.coefficients)) 

Coefficients: [-0.0003436622598304444,-2.7028946217444103,-3.1266371101423445e-10,0.00020196885521765493,-28.57518121293229,-0.0035443198285836615,-1.148271437106802,-1.408019904138137,0.27984272034633223,0.05025740009216324,-0.0010169937528697862,-0.00041071919113279414,0.9157052648296696,3.9822580516660384,0.9342110367211139,3.96760704505804,7.069634236467563,-1.4447964502428905e-05,-1.343380126652838e-05]


In [20]:
# Milestone 3: Construct Feature Importance to find top 10 features to choose to represent our dataset.
feature_importance = sorted(list(zip(new_df.columns[:-1], map(abs, coefficients))), key=lambda x: x[1], reverse=True)

print("Feature Importance:")
for feature, importance in feature_importance:
    print("  {}: {:.3f}".format(feature, importance))

Feature Importance:
  crsdiscount: 28.575
  policytermindicator: 7.070
  occupancytype: 3.982
  policycount: 3.968
  basementenclosurecrawlspacetype: 2.703
  hfiaasurcharge: 1.408
  federalpolicyfee: 1.148
  policycost: 0.934
  numberoffloorsininsuredbuilding: 0.916
  latitude: 0.280
  longitude: 0.050
  elevationdifference: 0.004
  lowestadjacentgrade: 0.001
  lowestfloorelevation: 0.000
  basefloodelevation: 0.000
  countycode: 0.000
  totalbuildinginsurancecoverage: 0.000
  totalcontentsinsurancecoverage: 0.000
  censustract: 0.000


# Construct 5 Linear Regression Models to find Ideal Range for Model Complexity
#### Using the Feature Importance model information, will start by training a Linear Regression Model with 1 feature (Top Feature Importance), then 5 features, 10 features, 15 features, and finally 19 features. There are a total 20 numerical features in the FEMA dataset, and the totalinsurancepremiumofthepolicy feature is the dependent variable. 

### Linear Regression Model with 1 Feature: crsdiscount

In [44]:
assembler_f1 = VectorAssembler(
    inputCols=['crsdiscount'], outputCol = 'predict_features_1')

new_df = assembler_f1.transform(new_df)

df_f1 = new_df.select('predict_features_1', 'totalinsurancepremiumofthepolicy')
train_data_f1, test_data_f1 = df_f1.randomSplit([0.7, 0.3], seed = 42)

lr_f1 = LinearRegression(featuresCol = 'predict_features_1', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f1', regParam = 0.3)
lr_model_f1 = lr_f1.fit(train_data_f1)

In [45]:
# RMSE for Train data
predictions_rmse_train_f1 = lr_model_f1.transform(train_data_f1)

evaluator_f1 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f1', metricName = 'rmse')
rmse_train_f1 = evaluator_f1.evaluate(predictions_rmse_train_f1)
print("Root Mean Squared Error (RMSE) on train data for Linear Regression using 1 feature: {:.7f}".format(rmse_train_f1))

Root Mean Squared Error (RMSE) on train data for Linear Regression using 1 feature: 1643.9152835


In [46]:
# R-squared for Train data
evaluator_r2_f1 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f1', metricName = 'r2')
r2_train_f1 = evaluator_r2_f1.evaluate(predictions_rmse_train_f1)
print("R-squared (R2) on train data for Linear Regression using 1 feature: {:.7f}".format(r2_train_f1))

R-squared (R2) on train data for Linear Regression using 1 feature: 0.0034574


In [47]:
# RMSE for Test data
predictions_rmse_test_f1 = lr_model_f1.transform(test_data_f1)

evaluator_f1 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f1', metricName = 'rmse')
rmse_test_f1 = evaluator_f1.evaluate(predictions_rmse_test_f1)
print("Root Mean Squared Error (RMSE) on test data for Linear Regression using 1 feature: {:.7f}".format(rmse_test_f1))

Root Mean Squared Error (RMSE) on test data for Linear Regression using 1 feature: 1652.3239030


In [48]:
# R-squared for Test data
evaluator_r2_f1 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f1', metricName = 'r2')
r2_test_f1 = evaluator_r2_f1.evaluate(predictions_rmse_test_f1)
print("R-squared (R2) on test data for Linear Regression using 1 feature: {:.7f}".format(r2_test_f1))

R-squared (R2) on test data for Linear Regression using 1 feature: 0.0033947


### Linear Regression Model with 5 Features: 
#### crsdiscount', 'policytermindicator', 'occupancytype', 'policycount', basementenclosurecrawlspacetype'

In [49]:
assembler_f5 = VectorAssembler(
    inputCols=['crsdiscount',
  'policytermindicator',
  'occupancytype',
  'policycount',
  'basementenclosurecrawlspacetype'], outputCol = 'predict_features_5')

new_df = assembler_f5.transform(new_df)

df_f5 = new_df.select('predict_features_5', 'totalinsurancepremiumofthepolicy')
train_data_f5, test_data_f5 = df_f5.randomSplit([0.7, 0.3], seed = 42)

lr_f5 = LinearRegression(featuresCol = 'predict_features_5', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f5', regParam = 0.3)
lr_model_f5 = lr_f5.fit(train_data_f5)

In [50]:
# RMSE for Train data
predictions_rmse_train_f5 = lr_model_f5.transform(train_data_f5)

evaluator_f5 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f5', metricName = 'rmse')
rmse_train_f5 = evaluator_f5.evaluate(predictions_rmse_train_f5)
print("Root Mean Squared Error (RMSE) on train data for Linear Regression using 5 features: {:.7f}".format(rmse_train_f5))

Root Mean Squared Error (RMSE) on train data for Linear Regression using 5 features: 1407.3976759


In [51]:
# R-squared for Train data
evaluator_r2_f5 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f5', metricName = 'r2')
r2_train_f5 = evaluator_r2_f5.evaluate(predictions_rmse_train_f5)
print("R-squared (R2) on train data for Linear Regression using 5 features: {:.7f}".format(r2_train_f5))

R-squared (R2) on train data for Linear Regression using 5 features: 0.2814601


In [52]:
# RMSE for Test data
predictions_rmse_test_f5 = lr_model_f5.transform(test_data_f5)

evaluator_f5 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f5', metricName = 'rmse')
rmse_test_f5 = evaluator_f5.evaluate(predictions_rmse_test_f5)
print("Root Mean Squared Error (RMSE) on test data for Linear Regression using 5 features: {:.7f}".format(rmse_test_f5))

Root Mean Squared Error (RMSE) on test data for Linear Regression using 5 features: 1375.8439908


In [53]:
# R-squared for Test data
evaluator_r2_f5 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f5', metricName = 'r2')
r2_test_f5 = evaluator_r2_f5.evaluate(predictions_rmse_test_f5)
print("R-squared (R2) on test data for Linear Regression using 5 features: {:.7f}".format(r2_test_f5))

R-squared (R2) on test data for Linear Regression using 5 features: 0.2815993


### Linear Regression Model with 10 Features:
#### 'basementenclosurecrawlspacetype', 'crsdiscount', 'federalpolicyfee', 'hfiaasurcharge', 'latitude', 'numberoffloorsininsuredbuilding', 'occupancytype', 'policycost', 'policycount', 'policytermindicator'

In [54]:
# Milestone 3: Construct Linear Regression prediction model with the new Chosen 10 Features
# f10 = LinReg model with top 10 features (chosen from Feature Importance)

assembler_f10 = VectorAssembler(
    inputCols=['basementenclosurecrawlspacetype',
 'crsdiscount',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator'],
    outputCol = 'predict_features_10')

new_df = assembler_f10.transform(new_df)

df_f10 = new_df.select('predict_features_10', 'totalinsurancepremiumofthepolicy')
train_data_f10, test_data_f10 = df_f10.randomSplit([0.7, 0.3], seed = 42)

lr_f10 = LinearRegression(featuresCol = 'predict_features_10', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f10', regParam = 0.3)
lr_model_f10 = lr_f10.fit(train_data_f10)

In [55]:
# Milestone 3: RMSE for Train data
# RMSE ranges from 0 to infinity, lower RMSE the better, (higher weight to larger errors)
predictions_rmse_train_f10 = lr_model_f10.transform(train_data_f10)

evaluator_f10 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f10', metricName = 'rmse')
rmse_train_f10 = evaluator_f10.evaluate(predictions_rmse_train_f10)
print("Root Mean Squared Error (RMSE) on train data for Linear Regression using 10 features: {:.7f}".format(rmse_train_f10))

Root Mean Squared Error (RMSE) on train data for Linear Regression using 10 features: 118.1325824


In [56]:
# R-squared for Train data
evaluator_r2_f10 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f10', metricName = 'r2')
r2_train_f10 = evaluator_r2_f10.evaluate(predictions_rmse_train_f10)
print("R-squared (R2) on train data for Linear Regression using 10 features: {:.7f}".format(r2_train_f10))

R-squared (R2) on train data for Linear Regression using 10 features: 0.9949013


In [57]:
# RMSE for Test data
predictions_rmse_test_f10 = lr_model_f10.transform(test_data_f10)

evaluator_f10 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f10', metricName = 'rmse')
rmse_test_f10 = evaluator_f10.evaluate(predictions_rmse_test_f10)
print("Root Mean Squared Error (RMSE) on test data for Linear Regression using 10 features: {:.7f}".format(rmse_test_f10))

Root Mean Squared Error (RMSE) on test data for Linear Regression using 10 features: 117.1184727


In [58]:
# R-squared for Test data
evaluator_r2_f10 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f10', metricName = 'r2')
r2_test_f10 = evaluator_r2_f10.evaluate(predictions_rmse_test_f10)
print("R-squared (R2) on test data for Linear Regression using 10 features: {:.7f}".format(r2_test_f10))

R-squared (R2) on test data for Linear Regression using 10 features: 0.9948833


### Linear Regression Model with 15 Features
#### 'crsdiscount', 'policytermindicator', 'occupancytype', 'policycount', 'basementenclosurecrawlspacetype', 'hfiaasurcharge', 'federalpolicyfee', 'policycost', 'numberoffloorsininsuredbuilding', 'latitude', 'longitude', 'elevationdifference', 'lowestadjacentgrade', 'lowestfloorelevation', 'basefloodelevation'

In [59]:
assembler_f15 = VectorAssembler(
    inputCols=['crsdiscount',
  'policytermindicator',
  'occupancytype',
  'policycount',
  'basementenclosurecrawlspacetype',
  'hfiaasurcharge',
  'federalpolicyfee',
  'policycost',
  'numberoffloorsininsuredbuilding',
  'latitude',
  'longitude',
  'elevationdifference',
  'lowestadjacentgrade',
  'lowestfloorelevation',
  'basefloodelevation'], outputCol = 'predict_features_15')

new_df = assembler_f15.transform(new_df)

df_f15 = new_df.select('predict_features_15', 'totalinsurancepremiumofthepolicy')
train_data_f15, test_data_f15 = df_f15.randomSplit([0.7, 0.3], seed = 42)

lr_f15 = LinearRegression(featuresCol = 'predict_features_15', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f15', regParam = 0.3)
lr_model_f15 = lr_f15.fit(train_data_f15)

In [60]:
# RMSE for Train data
predictions_rmse_train_f15 = lr_model_f15.transform(train_data_f15)

evaluator_f15 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f15', metricName = 'rmse')
rmse_train_f15 = evaluator_f15.evaluate(predictions_rmse_train_f15)
print("Root Mean Squared Error (RMSE) on train data for Linear Regression using 15 features: {:.7f}".format(rmse_train_f15))

Root Mean Squared Error (RMSE) on train data for Linear Regression using 15 features: 117.6225831


In [61]:
# R-squared for Train data
evaluator_r2_f15 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f15', metricName = 'r2')
r2_train_f15 = evaluator_r2_f15.evaluate(predictions_rmse_train_f15)
print("R-squared (R2) on train data for Linear Regression using 15 features: {:.7f}".format(r2_train_f15))

R-squared (R2) on train data for Linear Regression using 15 features: 0.9948743


In [62]:
# RMSE for Test data
predictions_rmse_test_f15 = lr_model_f15.transform(test_data_f15)

evaluator_f15 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f15', metricName = 'rmse')
rmse_test_f15 = evaluator_f15.evaluate(predictions_rmse_test_f15)
print("Root Mean Squared Error (RMSE) on test data for Linear Regression using 15 features: {:.7f}".format(rmse_test_f15))

Root Mean Squared Error (RMSE) on test data for Linear Regression using 15 features: 118.2675894


In [63]:
# R-squared for Test data
evaluator_r2_f15 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f15', metricName = 'r2')
r2_test_f15 = evaluator_r2_f15.evaluate(predictions_rmse_test_f15)
print("R-squared (R2) on test data for Linear Regression using 15 features: {:.7f}".format(r2_test_f15))

R-squared (R2) on test data for Linear Regression using 15 features: 0.9949487


### Linear Regression Model with 19 Features
#### 'crsdiscount', 'policytermindicator', 'occupancytype', 'policycount', 'basementenclosurecrawlspacetype', 'hfiaasurcharge', 'federalpolicyfee', 'policycost', 'numberoffloorsininsuredbuilding', 'latitude', 'longitude', 'elevationdifference', 'lowestadjacentgrade:', 'lowestfloorelevation', 'basefloodelevation', 'countycode', 'totalbuildinginsurancecoverage', 'totalcontentsinsurancecoverage', 'censustract'

In [64]:
# Check if 'predict_features_19' already exists, if so, drop it
if 'predict_features_19' in new_df.columns:
    new_df = new_df.drop('predict_features_19')

# Apply VectorAssembler
assembler_f19 = VectorAssembler(
    inputCols=[
        'crsdiscount', 'policytermindicator', 'occupancytype', 'policycount',
        'basementenclosurecrawlspacetype', 'hfiaasurcharge', 'federalpolicyfee',
        'policycost', 'numberoffloorsininsuredbuilding', 'latitude', 'longitude',
        'elevationdifference', 'lowestadjacentgrade', 'lowestfloorelevation',
        'basefloodelevation', 'countycode', 'totalbuildinginsurancecoverage',
        'totalcontentsinsurancecoverage', 'censustract'
    ],
    outputCol='predict_features_19'
)

new_df = assembler_f19.transform(new_df)

# Select the required columns
df_f19 = new_df.select('predict_features_19', 'totalinsurancepremiumofthepolicy')

# Split the data into training and testing sets
train_data_f19, test_data_f19 = df_f19.randomSplit([0.7, 0.3], seed=42)

# Define and fit the Linear Regression model
lr_f19 = LinearRegression(
    featuresCol='predict_features_19', 
    labelCol='totalinsurancepremiumofthepolicy', 
    predictionCol='predicted_premium_f19', 
    regParam=0.3
)
lr_model_f19 = lr_f19.fit(train_data_f19)

In [65]:
# RMSE for Train data
predictions_rmse_train_f19 = lr_model_f19.transform(train_data_f19)

evaluator_f19 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f19', metricName = 'rmse')
rmse_train_f19 = evaluator_f19.evaluate(predictions_rmse_train_f19)
print("Root Mean Squared Error (RMSE) on train data for Linear Regression using 19 features: {:.7f}".format(rmse_train_f19))

Root Mean Squared Error (RMSE) on train data for Linear Regression using 19 features: 117.5606982


In [66]:
# R-squared for Train data
evaluator_r2_f19 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f19', metricName = 'r2')
r2_train_f19 = evaluator_r2_f19.evaluate(predictions_rmse_train_f19)
print("R-squared (R2) on train data for Linear Regression using 19 features: {:.7f}".format(r2_train_f19))

R-squared (R2) on train data for Linear Regression using 19 features: 0.9948655


In [67]:
# RMSE for Test data
predictions_rmse_test_f19 = lr_model_f19.transform(test_data_f19)

evaluator_f19 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f19', metricName = 'rmse')
rmse_test_f19 = evaluator_f19.evaluate(predictions_rmse_test_f19)
print("Root Mean Squared Error (RMSE) on test data for Linear Regression using 19 features: {:.7f}".format(rmse_test_f19))

Root Mean Squared Error (RMSE) on test data for Linear Regression using 19 features: 118.0065623


In [68]:
# R-squared for Test data
evaluator_r2_f19 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_f19', metricName = 'r2')
r2_test_f19 = evaluator_r2_f19.evaluate(predictions_rmse_test_f19)
print("R-squared (R2) on test data for Linear Regression using 19 features: {:.7f}".format(r2_test_f19))

R-squared (R2) on test data for Linear Regression using 19 features: 0.9950025


## Note to group: exclude Bar plots when submitting for Milestone 4. This is incorrect; not a Fitting Graph.

In [ ]:
# Milestone 3: Bar plot with RMSE values, comparing train and test data
# To answer Question 4: "Where does your model fit in the fitting graph?"
df_barplot = pd.DataFrame({'RMSE': ['Train RMSE', 'Test RMSE'], 'Error Values': [118.133, 117.118]})  
df_barplot.plot.bar(x = 'RMSE', y = 'Error Values')  

In [ ]:
# Milestone 3: Bar plot with R-squared values, comparing train and test data
# To answer Question 4: "Where does your model fit in the fitting graph?"
df_barplot = pd.DataFrame({'R-Squared': ['Train R-Squared', 'Test R-Squared'], 'Values': [0.995, 0.995]})  
df_barplot.plot.bar(x = 'R-Squared', y = 'Values')  

### Scatterplots 

In [ ]:
changed_num_df = changed_df.sample(False, 0.1)
num_pdf = changed_num_df.toPandas()
print(num_pdf)

In [ ]:
plt.scatter(x=num_pdf['policycost'],y=num_pdf['totalinsurancepremiumofthepolicy'])
plt.xlabel('Policy Cost')
plt.ylabel('Total Insurance Premium of the Policy')          
plt.title('Policy Cost vs Total Insurance Premium of the Policy')
plt.show()

In [ ]:
plt.scatter(x=num_pdf['countycode'],y=num_pdf['censustract'])
plt.xlabel('County Code')
plt.ylabel('Census Tract') 
plt.title('County Code vs Census Tract')
plt.show()

In [ ]:
plt.scatter(x=num_pdf['policycount'],y=num_pdf['totalbuildinginsurancecoverage'])
plt.xlabel('Policy Count')
plt.ylabel('Total Building Insurance Coverage') 
plt.title('Policy Count vs Total Building Insurance Coverage')
plt.show()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


# Exploring Decision Tree Model

In [24]:
# creating train and test sets
assembler = VectorAssembler(
    inputCols=['basefloodelevation',
               'basementenclosurecrawlspacetype',
               'censustract',
               'countycode',
               'crsdiscount',
               'elevationdifference',
               'federalpolicyfee',
               'hfiaasurcharge',
               'latitude',
               'longitude',
               'lowestadjacentgrade',
               'lowestfloorelevation',
               'numberoffloorsininsuredbuilding',
               'occupancytype',
               'policycost',
               'policycount',
               'policytermindicator',
               'totalbuildinginsurancecoverage',
               'totalcontentsinsurancecoverage'],
    outputCol='new_features')

new_df = assembler.transform(new_df)
final_data = new_df.select('new_features', 'totalinsurancepremiumofthepolicy')

train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=42)

dt = DecisionTreeRegressor(featuresCol='new_features', labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium')
dt_model = dt.fit(train_data)

24/05/28 13:18:02 WARN MemoryStore: Not enough space to cache rdd_99_88 in memory! (computed 1026.9 KiB so far)
24/05/28 13:18:02 WARN MemoryStore: Not enough space to cache rdd_99_89 in memory! (computed 1026.9 KiB so far)
24/05/28 13:18:02 WARN BlockManager: Persisting block rdd_99_89 to disk instead.
24/05/28 13:18:02 WARN BlockManager: Persisting block rdd_99_88 to disk instead.
24/05/28 13:18:02 WARN MemoryStore: Not enough space to cache rdd_99_91 in memory! (computed 2.3 MiB so far)
24/05/28 13:18:02 WARN BlockManager: Persisting block rdd_99_91 to disk instead.
24/05/28 13:18:02 WARN MemoryStore: Not enough space to cache rdd_99_90 in memory! (computed 1544.0 KiB so far)
24/05/28 13:18:02 WARN BlockManager: Persisting block rdd_99_90 to disk instead.
24/05/28 13:18:03 WARN MemoryStore: Not enough space to cache rdd_99_84 in memory! (computed 2.3 MiB so far)
24/05/28 13:18:03 WARN MemoryStore: Not enough space to cache rdd_99_85 in memory! (computed 2.3 MiB so far)
24/05/28 13:1

In [25]:
# After fitting the DecisionTreeRegressor model
feature_importances = dt_model.featureImportances
print("Feature Importances: {}".format(feature_importances))

# Constructing Feature Importance to find top 10 features
feature_importance_list = list(zip(assembler.getInputCols(), feature_importances))
feature_importance_list_sorted = sorted(feature_importance_list, key=lambda x: x[1], reverse=True)

print("Top 10 Feature Importances:")
for feature, importance in feature_importance_list_sorted[:10]:
    print("  {}: {:.3f}".format(feature, importance))

Feature Importances: (19,[0,4,5,7,11,14,15,17,18],[0.017533895627270216,0.01191545429335759,0.027226298854630335,0.004125763495195851,0.004570672369236896,0.7319316313274444,0.16124047562678448,0.019769204962302646,0.021686603443777562])
Top 10 Feature Importances:
  policycost: 0.732
  policycount: 0.161
  elevationdifference: 0.027
  totalcontentsinsurancecoverage: 0.022
  totalbuildinginsurancecoverage: 0.020
  basefloodelevation: 0.018
  crsdiscount: 0.012
  lowestfloorelevation: 0.005
  hfiaasurcharge: 0.004
  basementenclosurecrawlspacetype: 0.000


In [32]:
print("Top 19 Feature Importances:")
for feature, importance in feature_importance_list_sorted[:19]:
    print("  {}: {:.3f}".format(feature, importance))

Top 19 Feature Importances:
  policycost: 0.732
  policycount: 0.161
  elevationdifference: 0.027
  totalcontentsinsurancecoverage: 0.022
  totalbuildinginsurancecoverage: 0.020
  basefloodelevation: 0.018
  crsdiscount: 0.012
  lowestfloorelevation: 0.005
  hfiaasurcharge: 0.004
  basementenclosurecrawlspacetype: 0.000
  censustract: 0.000
  countycode: 0.000
  federalpolicyfee: 0.000
  latitude: 0.000
  longitude: 0.000
  lowestadjacentgrade: 0.000
  numberoffloorsininsuredbuilding: 0.000
  occupancytype: 0.000
  policytermindicator: 0.000


# Construct 5 Decision Trees Models to find the Most Ideal Model 

### Using the Feature Importance model information, will start by training a Decision Trees Model with 1 feature (Top Feature Importance), then 5 features, 10 features, 15 features, and finally 19 features. There are a total 20 numerical features in the FEMA dataset, and the totalinsurancepremiumofthepolicy feature is the dependent variable.

### 1. Decision Trees Model with 1 Feature: 
policycost

In [26]:
assembler_f1 = VectorAssembler(
    inputCols=['policycost'], outputCol='predict_features_1')

new_df = assembler_f1.transform(new_df)

df_f1 = new_df.select('predict_features_1', 'totalinsurancepremiumofthepolicy')
train_data_f1, test_data_f1 = df_f1.randomSplit([0.7, 0.3], seed=42)

dt_f1 = DecisionTreeRegressor(featuresCol='predict_features_1', labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f1')
dt_model_f1 = dt_f1.fit(train_data_f1)

# RMSE for Train data
predictions_rmse_train_f1 = dt_model_f1.transform(train_data_f1)

evaluator_f1 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f1', metricName='rmse')
rmse_train_f1 = evaluator_f1.evaluate(predictions_rmse_train_f1)
print("Root Mean Squared Error (RMSE) on train data for Decision Tree using 1 feature: {:.7f}".format(rmse_train_f1))

# R-squared for Train data
evaluator_r2_f1 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f1', metricName='r2')
r2_train_f1 = evaluator_r2_f1.evaluate(predictions_rmse_train_f1)
print("R-squared (R2) on train data for Decision Tree using 1 feature: {:.7f}".format(r2_train_f1))

# RMSE for Test data
predictions_rmse_test_f1 = dt_model_f1.transform(test_data_f1)

evaluator_f1 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f1', metricName='rmse')
rmse_test_f1 = evaluator_f1.evaluate(predictions_rmse_test_f1)
print("Root Mean Squared Error (RMSE) on test data for Decision Tree using 1 feature: {:.7f}".format(rmse_test_f1))

# R-squared for Test data
evaluator_r2_f1 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f1', metricName='r2')
r2_test_f1 = evaluator_r2_f1.evaluate(predictions_rmse_test_f1)
print("R-squared (R2) on test data for Decision Tree using 1 feature: {:.7f}".format(r2_test_f1))


Root Mean Squared Error (RMSE) on train data for Decision Tree using 1 feature: 1292.9460710


R-squared (R2) on train data for Decision Tree using 1 feature: 0.3869786


Root Mean Squared Error (RMSE) on test data for Decision Tree using 1 feature: 1280.3065343


R-squared (R2) on test data for Decision Tree using 1 feature: 0.3938157


### 2. Decision Trees Model with 5 Feature: 
policycost, policycount, elevationdifference, totalcontentsinsurancecoverage,totalbuildinginsurancecoverage

In [28]:
assembler_f5 = VectorAssembler(
    inputCols=['policycost',
               'policycount',
               'elevationdifference',
               'totalcontentsinsurancecoverage',
               'totalbuildinginsurancecoverage'], outputCol='predict_features_5')

new_df = assembler_f5.transform(new_df)

df_f5 = new_df.select('predict_features_5', 'totalinsurancepremiumofthepolicy')
train_data_f5, test_data_f5 = df_f5.randomSplit([0.7, 0.3], seed=42)

dt_f5 = DecisionTreeRegressor(featuresCol='predict_features_5', labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f5')
dt_model_f5 = dt_f5.fit(train_data_f5)

# RMSE for Train data
predictions_rmse_train_f5 = dt_model_f5.transform(train_data_f5)

evaluator_f5 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f5', metricName='rmse')
rmse_train_f5 = evaluator_f5.evaluate(predictions_rmse_train_f5)
print("Root Mean Squared Error (RMSE) on train data for Decision Tree using 5 features: {:.7f}".format(rmse_train_f5))

# R-squared for Train data
evaluator_r2_f5 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f5', metricName='r2')
r2_train_f5 = evaluator_r2_f5.evaluate(predictions_rmse_train_f5)
print("R-squared (R2) on train data for Decision Tree using 5 features: {:.7f}".format(r2_train_f5))

# RMSE for Test data
predictions_rmse_test_f5 = dt_model_f5.transform(test_data_f5)

evaluator_f5 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f5', metricName='rmse')
rmse_test_f5 = evaluator_f5.evaluate(predictions_rmse_test_f5)
print("Root Mean Squared Error (RMSE) on test data for Decision Tree using 5 features: {:.7f}".format(rmse_test_f5))

# R-squared for Test data
evaluator_r2_f5 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f5', metricName='r2')
r2_test_f5 = evaluator_r2_f5.evaluate(predictions_rmse_test_f5)
print("R-squared (R2) on test data for Decision Tree using 5 features: {:.7f}".format(r2_test_f5))


Root Mean Squared Error (RMSE) on train data for Decision Tree using 5 features: 1152.8592878


R-squared (R2) on train data for Decision Tree using 5 features: 0.5126214


Root Mean Squared Error (RMSE) on test data for Decision Tree using 5 features: 1137.6243066


R-squared (R2) on test data for Decision Tree using 5 features: 0.5213957


### 3. Decision Trees Model with 10 Feature: 
policycost, policycount, elevationdifference, totalcontentsinsurancecoverage, totalbuildinginsurancecoverage, basefloodelevation, crsdiscount, lowestfloorelevation, hfiaasurcharge, basementenclosurecrawlspacetype

In [29]:
assembler_f10 = VectorAssembler(
    inputCols=['policycost',
               'policycount',
               'elevationdifference',
               'totalcontentsinsurancecoverage',
               'totalbuildinginsurancecoverage',
               'basefloodelevation',
               'crsdiscount',
               'lowestfloorelevation',
               'hfiaasurcharge',
               'basementenclosurecrawlspacetype'], outputCol='predict_features_10')

new_df = assembler_f10.transform(new_df)

df_f10 = new_df.select('predict_features_10', 'totalinsurancepremiumofthepolicy')
train_data_f10, test_data_f10 = df_f10.randomSplit([0.7, 0.3], seed=42)

dt_f10 = DecisionTreeRegressor(featuresCol='predict_features_10', labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f10')
dt_model_f10 = dt_f10.fit(train_data_f10)

# RMSE for Train data
predictions_rmse_train_f10 = dt_model_f10.transform(train_data_f10)

evaluator_f10 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f10', metricName='rmse')
rmse_train_f10 = evaluator_f10.evaluate(predictions_rmse_train_f10)
print("Root Mean Squared Error (RMSE) on train data for Decision Tree using 10 features: {:.7f}".format(rmse_train_f10))

# R-squared for Train data
evaluator_r2_f10 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f10', metricName='r2')
r2_train_f10 = evaluator_r2_f10.evaluate(predictions_rmse_train_f10)
print("R-squared (R2) on train data for Decision Tree using 10 features: {:.7f}".format(r2_train_f10))

# RMSE for Test data
predictions_rmse_test_f10 = dt_model_f10.transform(test_data_f10)

evaluator_f10 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f10', metricName='rmse')
rmse_test_f10 = evaluator_f10.evaluate(predictions_rmse_test_f10)
print("Root Mean Squared Error (RMSE) on test data for Decision Tree using 10 features: {:.7f}".format(rmse_test_f10))

# R-squared for Test data
evaluator_r2_f10 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f10', metricName='r2')
r2_test_f10 = evaluator_r2_f10.evaluate(predictions_rmse_test_f10)
print("R-squared (R2) on test data for Decision Tree using 10 features: {:.7f}".format(r2_test_f10))


Root Mean Squared Error (RMSE) on train data for Decision Tree using 10 features: 1141.8630778


R-squared (R2) on train data for Decision Tree using 10 features: 0.5219198


Root Mean Squared Error (RMSE) on test data for Decision Tree using 10 features: 1129.1083224


R-squared (R2) on test data for Decision Tree using 10 features: 0.5284291


### 4. Decision Trees Model with 15 Feature:
policycost, policycount, elevationdifference, totalcontentsinsurancecoverage, totalbuildinginsurancecoverage, basefloodelevation, crsdiscount, lowestfloorelevation, hfiaasurcharge, basementenclosurecrawlspacetype, censustract, countycode, federalpolicyfee, latitude, longitude








In [30]:
assembler_f15 = VectorAssembler(
    inputCols=['policycost',
               'policycount',
               'elevationdifference',
               'totalcontentsinsurancecoverage',
               'totalbuildinginsurancecoverage',
               'basefloodelevation',
               'crsdiscount',
               'lowestfloorelevation',
               'hfiaasurcharge',
               'basementenclosurecrawlspacetype',
               'censustract',
               'countycode',
               'federalpolicyfee',
               'latitude',
               'longitude'], outputCol='predict_features_15')

new_df = assembler_f15.transform(new_df)

df_f15 = new_df.select('predict_features_15', 'totalinsurancepremiumofthepolicy')
train_data_f15, test_data_f15 = df_f15.randomSplit([0.7, 0.3], seed=42)

dt_f15 = DecisionTreeRegressor(featuresCol='predict_features_15', labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f15')
dt_model_f15 = dt_f15.fit(train_data_f15)

# RMSE for Train data
predictions_rmse_train_f15 = dt_model_f15.transform(train_data_f15)

evaluator_f15 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f15', metricName='rmse')
rmse_train_f15 = evaluator_f15.evaluate(predictions_rmse_train_f15)
print("Root Mean Squared Error (RMSE) on train data for Decision Tree using 15 features: {:.7f}".format(rmse_train_f15))

# R-squared for Train data
evaluator_r2_f15 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f15', metricName='r2')
r2_train_f15 = evaluator_r2_f15.evaluate(predictions_rmse_train_f15)
print("R-squared (R2) on train data for Decision Tree using 15 features: {:.7f}".format(r2_train_f15))

# RMSE for Test data
predictions_rmse_test_f15 = dt_model_f15.transform(test_data_f15)

evaluator_f15 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f15', metricName='rmse')
rmse_test_f15 = evaluator_f15.evaluate(predictions_rmse_test_f15)
print("Root Mean Squared Error (RMSE) on test data for Decision Tree using 15 features: {:.7f}".format(rmse_test_f15))

# R-squared for Test data
evaluator_r2_f15 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f15', metricName='r2')
r2_test_f15 = evaluator_r2_f15.evaluate(predictions_rmse_test_f15)
print("R-squared (R2) on test data for Decision Tree using 15 features: {:.7f}".format(r2_test_f15))


24/05/28 13:53:11 WARN MemoryStore: Not enough space to cache rdd_367_89 in memory! (computed 41.6 MiB so far)
24/05/28 13:53:11 WARN MemoryStore: Not enough space to cache rdd_367_91 in memory! (computed 17.6 MiB so far)
24/05/28 13:53:11 WARN BlockManager: Persisting block rdd_367_89 to disk instead.
24/05/28 13:53:11 WARN BlockManager: Persisting block rdd_367_91 to disk instead.
24/05/28 13:53:11 WARN MemoryStore: Not enough space to cache rdd_367_94 in memory! (computed 41.6 MiB so far)
24/05/28 13:53:11 WARN BlockManager: Persisting block rdd_367_94 to disk instead.
24/05/28 13:53:11 WARN MemoryStore: Not enough space to cache rdd_367_95 in memory! (computed 26.6 MiB so far)
24/05/28 13:53:11 WARN BlockManager: Persisting block rdd_367_95 to disk instead.
24/05/28 13:53:11 WARN MemoryStore: Not enough space to cache rdd_367_92 in memory! (computed 41.6 MiB so far)
24/05/28 13:53:11 WARN BlockManager: Persisting block rdd_367_92 to disk instead.
24/05/28 13:53:11 WARN MemoryStore:

Root Mean Squared Error (RMSE) on train data for Decision Tree using 15 features: 1146.1381342


R-squared (R2) on train data for Decision Tree using 15 features: 0.5182932


Root Mean Squared Error (RMSE) on test data for Decision Tree using 15 features: 1132.0465987


R-squared (R2) on test data for Decision Tree using 15 features: 0.5260645


### 5. Decision Trees Model with 19 Feature:
policycost, policycount, elevationdifference, totalcontentsinsurancecoverage, totalbuildinginsurancecoverage, basefloodelevation, crsdiscount, lowestfloorelevation, hfiaasurcharge, basementenclosurecrawlspacetype, censustract, countycode, federalpolicyfee, latitude, longitude, lowestadjacentgrade, numberoffloorsininsuredbuilding, occupancytype, policytermindicator

In [31]:
assembler_f19 = VectorAssembler(
    inputCols=['policycost',
               'policycount',
               'elevationdifference',
               'totalcontentsinsurancecoverage',
               'totalbuildinginsurancecoverage',
               'basefloodelevation',
               'crsdiscount',
               'lowestfloorelevation',
               'hfiaasurcharge',
               'basementenclosurecrawlspacetype',
               'censustract',
               'countycode',
               'federalpolicyfee',
               'latitude',
               'longitude',
               'lowestadjacentgrade',
               'numberoffloorsininsuredbuilding',
               'occupancytype',
               'policytermindicator'], outputCol='predict_features_19')

new_df = assembler_f19.transform(new_df)

df_f19 = new_df.select('predict_features_19', 'totalinsurancepremiumofthepolicy')
train_data_f19, test_data_f19 = df_f19.randomSplit([0.7, 0.3], seed=42)

dt_f19 = DecisionTreeRegressor(featuresCol='predict_features_19', labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f19')
dt_model_f19 = dt_f19.fit(train_data_f19)

# RMSE for Train data
predictions_rmse_train_f19 = dt_model_f19.transform(train_data_f19)

evaluator_f19 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f19', metricName='rmse')
rmse_train_f19 = evaluator_f19.evaluate(predictions_rmse_train_f19)
print("Root Mean Squared Error (RMSE) on train data for Decision Tree using 19 features: {:.7f}".format(rmse_train_f19))

# R-squared for Train data
evaluator_r2_f19 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f19', metricName='r2')
r2_train_f19 = evaluator_r2_f19.evaluate(predictions_rmse_train_f19)
print("R-squared (R2) on train data for Decision Tree using 19 features: {:.7f}".format(r2_train_f19))

# RMSE for Test data
predictions_rmse_test_f19 = dt_model_f19.transform(test_data_f19)

evaluator_f19 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f19', metricName='rmse')
rmse_test_f19 = evaluator_f19.evaluate(predictions_rmse_test_f19)
print("Root Mean Squared Error (RMSE) on test data for Decision Tree using 19 features: {:.7f}".format(rmse_test_f19))

# R-squared for Test data
evaluator_r2_f19 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium_f19', metricName='r2')
r2_test_f19 = evaluator_r2_f19.evaluate(predictions_rmse_test_f19)
print("R-squared (R2) on test data for Decision Tree using 19 features: {:.7f}".format(r2_test_f19))


24/05/28 14:03:48 WARN MemoryStore: Not enough space to cache rdd_446_86 in memory! (computed 29.5 MiB so far)
24/05/28 14:03:48 WARN MemoryStore: Not enough space to cache rdd_446_84 in memory! (computed 29.5 MiB so far)
24/05/28 14:03:48 WARN BlockManager: Persisting block rdd_446_84 to disk instead.
24/05/28 14:03:48 WARN BlockManager: Persisting block rdd_446_86 to disk instead.
24/05/28 14:03:48 WARN MemoryStore: Not enough space to cache rdd_446_82 in memory! (computed 29.5 MiB so far)
24/05/28 14:03:48 WARN BlockManager: Persisting block rdd_446_82 to disk instead.
24/05/28 14:03:48 WARN MemoryStore: Not enough space to cache rdd_446_87 in memory! (computed 29.5 MiB so far)
24/05/28 14:03:48 WARN BlockManager: Persisting block rdd_446_87 to disk instead.
24/05/28 14:03:48 WARN MemoryStore: Not enough space to cache rdd_446_85 in memory! (computed 29.5 MiB so far)
24/05/28 14:03:48 WARN BlockManager: Persisting block rdd_446_85 to disk instead.
24/05/28 14:03:48 WARN MemoryStore:

Root Mean Squared Error (RMSE) on train data for Decision Tree using 19 features: 1142.1079771


R-squared (R2) on train data for Decision Tree using 19 features: 0.5216784


Root Mean Squared Error (RMSE) on test data for Decision Tree using 19 features: 1127.5757127


R-squared (R2) on test data for Decision Tree using 19 features: 0.5297924


In [70]:
results = [
    ['Linear Regression', 'predict_features_1', 1643.9153, 0.0035, 1652.3239, 0.0034],
    ['Linear Regression', 'predict_features_5', 1407.3977, 0.2815, 1375.8440, 0.2816],
    ['Linear Regression', 'predict_features_10', 118.1326, 0.9949, 117.1185, 0.9949],
    ['Linear Regression', 'predict_features_15', 117.6226, 0.9949, 118.2676, 0.9949],
    ['Linear Regression', 'predict_features_19', 117.5607, 0.9949, 118.0066, 0.9950],
    ['Decision Tree', 'predict_features_1', 1292.9461, 0.3870, 1280.3065, 0.3938],
    ['Decision Tree', 'predict_features_5', 1152.8593, 0.5126, 1137.6243, 0.5214],
    ['Decision Tree', 'predict_features_10', 1141.8631, 0.5219, 1129.1083, 0.5284],
    ['Decision Tree', 'predict_features_15', 1146.1381, 0.5183, 1132.0466, 0.5261],
    ['Decision Tree', 'predict_features_19', 1142.1080, 0.5217, 1127.5757, 0.5298]
]

# Create a DataFrame for the results
results_df = pd.DataFrame(results, columns=['Model', 'Features', 'Train RMSE', 'Train R2', 'Test RMSE', 'Test R2'])

# Save the DataFrame to an Excel file
results_df.to_excel('model_comparison_results.xlsx', index=False)

print("Results have been saved to 'model_comparison_results.xlsx'")


Results have been saved to 'model_comparison_results.xlsx'


# Interactive Map - Showcasing Geography, Flood Risk and Insurance Premium

In [91]:
# Understading flood zone category

unique_floodzones = df.select("floodzone").distinct().collect()
for row in unique_floodzones:
    print(row['floodzone'])


A23
V24
AOB
A01
A19
AHB
A16
V19
A21
A17
A18
V27
A22
V06
V22
V21
V08
B
V14
V11
V18
V05
V16
A11
A27
A99
V
V04
A09
D
V23
A03
VE
V20
A25
V07
C
A26
V02
V15
AE
A10
A
V13
A20
X
A28
V09
V03
V12
A15
A30
V17
A14
A24
A0B
A08
V01
A06
AH
A07
A12
V30
V10
A04
A02
AR
A13
A05
AO
A29
A00
V29
V28
ALT
*
V8
A0
EMG
V9
ARE
X 0
A E
AO8
E
00X
None


In [89]:
# Add the new column with risk categories base of flood zone. 
# Reference: https://www.floodsmart.gov/flood-zones-and-maps

df = df.withColumn('risk_category',
                   F.when(F.col('floodzone').startswith('B'), 'Low Risk')
                    .when(F.col('floodzone').startswith('X'), 'Low Risk')
                    .when(F.col('floodzone').startswith('C'), 'Moderate Risk')
                    .when(F.col('floodzone').startswith('A'), 'High Risk')
                    .when(F.col('floodzone').startswith('V'), 'Coastal High Risk')
                    .when(F.col('floodzone').startswith('VE'), 'Coastal High Risk')
                    .when(F.col('floodzone').startswith('D'), 'Undetermined Risk')
                    .otherwise('Unknown Risk'))

# Show the result
df.select('floodzone', 'risk_category').show()

+---------+-------------+
|floodzone|risk_category|
+---------+-------------+
|        X|     Low Risk|
|       AE|    High Risk|
|        X|     Low Risk|
|       AE|    High Risk|
|      A10|    High Risk|
|      A02|    High Risk|
|      A06|    High Risk|
|        X|     Low Risk|
|        X|     Low Risk|
|        X|     Low Risk|
|      A08|    High Risk|
|       AE|    High Risk|
|       AE|    High Risk|
|       AE|    High Risk|
|        C|Moderate Risk|
|        X|     Low Risk|
|       AE|    High Risk|
|        C|Moderate Risk|
|        B|     Low Risk|
|        X|     Low Risk|
+---------+-------------+
only showing top 20 rows



In [90]:
df.printSchema()

root
 |-- agriculturestructureindicator: string (nullable = true)
 |-- basefloodelevation: double (nullable = true)
 |-- basementenclosurecrawlspacetype: integer (nullable = true)
 |-- cancellationdateoffloodpolicy: date (nullable = true)
 |-- censustract: long (nullable = true)
 |-- condominiumindicator: string (nullable = true)
 |-- construction: string (nullable = true)
 |-- countycode: integer (nullable = true)
 |-- crsdiscount: double (nullable = true)
 |-- deductibleamountinbuildingcoverage: string (nullable = true)
 |-- deductibleamountincontentscoverage: string (nullable = true)
 |-- elevatedbuildingindicator: string (nullable = true)
 |-- elevationcertificateindicator: string (nullable = true)
 |-- elevationdifference: integer (nullable = true)
 |-- federalpolicyfee: integer (nullable = true)
 |-- floodzone: string (nullable = true)
 |-- hfiaasurcharge: integer (nullable = true)
 |-- houseofworshipindicator: string (nullable = true)
 |-- latitude: double (nullable = true)
 |--

In [94]:
# Select relevant columns for the interactive map
df_map = df.select(
    'latitude',
    'longitude',
    'floodzone',
    'risk_category',
    'totalinsurancepremiumofthepolicy')


In [95]:
# Show the new DataFrame schema and some data
df_map.printSchema()
df_map.show(10, truncate=False)

# Optionally, cache the new DataFrame if you plan to use it multiple times
df_map.cache()

root
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- floodzone: string (nullable = true)
 |-- risk_category: string (nullable = false)
 |-- totalinsurancepremiumofthepolicy: integer (nullable = true)

+--------+---------+---------+-------------+--------------------------------+
|latitude|longitude|floodzone|risk_category|totalinsurancepremiumofthepolicy|
+--------+---------+---------+-------------+--------------------------------+
|43.3    |-71.8    |X        |Low Risk     |375                             |
|30.5    |-91.0    |AE       |High Risk    |280                             |
|33.7    |-79.0    |X        |Low Risk     |335                             |
|34.0    |-86.0    |AE       |High Risk    |916                             |
|26.0    |-80.1    |A10      |High Risk    |1288                            |
|32.8    |-80.0    |A02      |High Risk    |489                             |
|39.7    |-74.2    |A06      |High Risk    |1372           

DataFrame[latitude: double, longitude: double, floodzone: string, risk_category: string, totalinsurancepremiumofthepolicy: int]

In [97]:
pip install folium


     |████████████████████████████████| 100 kB 4.0 MB/s ta 0:00:011
     |████████████████████████████████| 81 kB 603 kB/s  eta 0:00:01
     |████████████████████████████████| 133 kB 11.5 MB/s eta 0:00:01
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Assuming df_map is already available as a DataFrame with relevant columns
df_map_pd = df_map.toPandas()

# Create a map centered around the average latitude and longitude
map_center = [df_map_pd['latitude'].mean(), df_map_pd['longitude'].mean()]
interactive_map = folium.Map(location=map_center, zoom_start=6)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(interactive_map)

# Function to determine marker color based on risk category
def get_marker_color(risk_category):
    if risk_category == 'Low Risk':
        return 'green'
    elif risk_category == 'Moderate Risk':
        return 'blue'
    elif risk_category == 'High Risk':
        return 'orange'
    elif risk_category == 'Coastal High Risk':
        return 'red'
    elif risk_category == 'Undetermined Risk':
        return 'gray'
    else:
        return 'black'

# Add markers to the map
for idx, row in df_map_pd.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Flood Zone: {row['floodzone']}<br>Risk Category: {row['risk_category']}<br>Insurance Premium: ${row['totalinsurancepremiumofthepolicy']}",
        icon=folium.Icon(color=get_marker_color(row['risk_category']))
    ).add_to(marker_cluster)

# Save the map to an HTML file
interactive_map.save('interactive_map.html')

# Display the map
interactive_map
